# 21. SQL을 이용해 DB와 대화해보자

## 1. 들어가며

### 파이썬과 데이터베이스가 만난다면?
---
파이썬에서 DB에 접근하기 위한 방법 중 가장 많이 사용하게 되는 것은 Python Database API(DB-API) 이다.

Python DB-API는 여러 DB에 접근할 수 있는 표준 API 입니다. 표준 API는 크게 3가지의 작업을 한다.

1. DB를 연결한다.
2. SQL문을 실행한다.
3. DB 연결을 닫는다.

파이썬 DB-API는 기본적으로 PEP249 인터페이스를 따르도록 권장된다. 여기서 기본적으로 선언되어 있는 함수들로는 `connect()`, `close()`, `commit()`, `rollback()` 등 여러 가지가 있다. 이 함수들을 이용하면 거의 모든 데이터베이스(DB)에 대해 동일한 함수를 사용해 조작할 수 있다.

[PEP 249 -- Python Database API Specification v2.0](https://legacy.python.org/dev/peps/pep-0249/)

파이썬에서 지원하는 DB는 매우 다양하기 때문에, 각 DB에 상응하는 별도의 DB 모듈을 다운받아야 한다. 수많은 DB 모듈이 존재하나, 이들 대부분이 Python DB-API 표준을 따르고 있으므로 동일한 API로 여러 DB를 사용할 수 있다.

파이썬은 MySQL, PostgreSQL, MSSQL, Sqlite, Oracle, Sybase, informix, mSQL 등 대표적인 DB를 모두 Python DB-API를 통해 지원한다.

<img src="./image/db-api.png" alt="DB-API" />

### 학습 목표
---
1. 기본적인 SQL문을 공부해 본다.
2. 파이썬을 통해 DB를 연결하여 SQL 질의를 시도해 본다.

### 배울 내용
---
1. 파이썬과 데이터베이스가 만난다면?
2. 파이썬의 DB 다루기
    - SQLite
    - Python & DB
    - Python DB API
    - DDL 문으로 테이블 생성하기
3. SQL
    - SQL의 기본
    - DISTINCT와 GROUP BY
    - 데이터 TYPE
    - 다양한 조건으로 조회하기
    - JOIN 수행하기

## 2. 파이썬으로 DB 다루기

### SQLite
---
__SQLite__는 서버의 필요 없이 DB의 파일에 기초하여 DB 처리를 구현한 __임베디드 SQL DB 엔진__이다. SQLite는 별도의 설치없이, 쉽고 편리하게 사용할 수 있다는 점에서 많이 사용되고 있다. Mac OS X나 Linux에는 SQLite가 기본적으로 설치되어 있다. 파이썬은 버전 2.5 이상일 경우 SQLite 사용을 위한 모듈이 기본 내장되어 있다.

### 파이썬과 DB
---
파이썬과 DB를 서로 다른 대륙에 살고 있는 사람이라고 생각해 보겠다. 파이썬 대륙과 DB 대륙에 있는 사람이 함께 이야기를 하려면 어떻게 해야 할까? 전화, 이메일과 같은 방법들을 이용해 시도해볼 수 있다. 이처럼 파이썬과 DB는 sqlite3라는 전화 중계기를 이용해 대화를 한다.

<img src="./image/SQLite3.png" alt="SQLite3" />

### Python DB API
---
파이썬에서 DB 연결을 위한 전화선인 __sqlite3__ 모듈을 import해보자.

In [1]:
import sqlite3

sqlite3 모듈은 파이썬 표준 라이브러리로 SQLite에 대한 인터페이스를 기본적으로 제공한다. 표준 라이브러리란 파이썬을 처음 설치할 때 기본적으로 설치되는 모듈이다. 정리하면, 표준 라이브러리인 sqlite3로 DB를 쉽게 이용할 수 있다.

작업디렉토리를 구성하자.

```bash
$ mkdir -p ~/aiffel/sql_to_db/sqlite
```

이제 파이썬과 DB를 연결해 보자. `conn`에 DB 이름을 정하여 입력한다. 

난 `mydb`로 지었다. `[이름].[확장자명]`의 형태로 저장하면 된다.

In [2]:
import os
db_path = os.getenv('HOME')+'/aiffel/sql_to_db/sqlite/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
print(conn)

`conn` 객체에는 SQL 연결과 관련된 셋팅이 포함되어 있다. 이번에는 `Connect()` 함수의 연결을 사용하는 새로운 Cursor 객체를 만든다.

In [3]:
c = conn.cursor()
print(c)

Cursor는 SQL 질의(Query)를 수행하고 결과를 얻는데 사용하는 객체이다. `INSERT`처럼 DB에만 적용되는 명령어를 사용한다면 Cursor를 안 사용할 수 있지만 `SELECT`와 같이 데이터를 불러올 때는 SQL 질의 수행 결과에 접근하기 위한 Cursor가 반드시 필요하다. 이러한 이유로 습관적으로 `conn.cursor()`를 사용하는 것을 권한다. 참고로, 파이썬 공식 개발 가이드에서도 Cursor를 사용하는 것을 표준으로 안내하고 있다.

>*`SQL INSERT문` : 테이블에 데이터를 삽입(추가)하는 질의이다. <br>
>`SQL SELECT문` : 테이블에 데이터를 조건에 따라 조회하는 질의이다.*

이제 SQL문을 실행해보자. 질의의 실행은 `execute()`를 이용하게 된다.

In [4]:
# stocks이라는 이름의 테이블을 하나 생성합니다. 혹시 이미 생성되었다면 생략합니다. 
c.execute("CREATE TABLE  IF NOT EXISTS  stocks (date text, trans text, symbol text, qty real, price real)")

# stocks 테이블에 데이터를 하나 인서트합니다. 
c.execute("INSERT INTO stocks VALUES ('20200701', 'TEST', 'AIFFEL', 1, 10000)")

# 방금 인서트한 데이터를 조회해 봅니다.
c.execute("SELECT * FROM stocks")

# 조회된 내역을 커서를 통해 가져와 출력해 봅니다.
print(c.fetchone())

('20200701', 'TEST', 'AIFFEL', 1.0, 10000.0)


방금 sqlite3 모듈을 이용해 데이터베이스에 테이블을 하나 만들고 데이터를 인서트한 후, 그 데이터를 조회해 보는 아주 기본적인 DB 핸들링 시나리오를 수행해 보았다.

### sqlite DB Browser
---
하지만 데이터베이스를 다루는 방법에는 sqlite3와 같은 Python DB API만 존재하는 것은 아니다. 이제 DB 브라우저 어플리케이션을 통해 데이터베이스에 직접 접근해서 질의를 수행해 볼 것이다. 우분투 환경에서 터미널을 열어 아래와 같이 설치를 진행하자.

```bash
$ sudo add-apt-repository -y ppa:linuxgndu/sqlitebrowser
$ sudo apt-get update
$ sudo apt-get install sqlitebrowser
```

설치가 완료되었다면 아래 그림과 같이 `sqlite`로 검색해서 나오는 DB Browser를 실행하자.

<img src="./image/sqlite.png" alt="sqlite" />

정상적으로 실행되었다면 아래와 같은 화면이 뜰 것이다. `데이터베이스 열기(O)` 버튼을 클릭 후 위에서 생성했던 `~/aiffel/sql_to_db/sqlite/mydb.db` 파일을 찾아서 열어보자. 여기까지 정상적으로 진행되었다면 아래 그림과 같이 `데이터베이스 구조` 탭에 `stocks`라는 이름의 테이블이 생성되었음을 확인할 수 있을 것이다. 이것으로 DB Browser는 Sqlite 데이터베이스를 관리할 준비를 마쳤다.

<img src="./image/sqlite2.png" alt="sqlite" />

그런데 이상한 것이 있다. 아까 우리는 `stocks` 테이블에 데이터를 하나 인서트했기 때문에 그 데이터를 확인해 보려고 데이터 보기 탭을 눌러 보았지만, 아무런 데이터도 조회되지 않는다. `SQL 실행` 탭에서 위에 수행했던 `SELECT * FROM stocks` 조회 쿼리를 수행해 봐도 결과는 동일하다. 무슨 일이 있었던 걸까?

삽입, 갱신, 삭제 등의 SQL 질의가 끝났다면 `conn.commit()`를 호출해야 DB가 실제로 업데이트된다. `commit()`을 하기 전에는 DB에 데이터가 업데이트된 것 같아 보여도 임시로만 바뀐 것이니 주의해야 한다. 하지만 우리는 sqlite3를 이용해 데이터가 잘 인서트되었음을 `SELECT`문의 결과를 통해서 이미 확인했다. 이건 또 무엇일까?

이것이 데이터베이스를 사용할 때의 주의사항이다. 우리는 sqlite3를 이용해 데이터베이스에 `connection`을 하나 맺었다. 이후 해당 `connection`을 통해 인서트된 데이터는 `conn.commit()`를 호출하기 전까지는 그 `connection` 안에서만 유효하다. 그래서 sqlite3에서는 `SELECT`가 되었지만, DB Browser 안에서는 조회되지 않는 것이다.

그럼 원본 데이터에 실제로 적용하려면 어떻게 해야 할까? 아래와 같이 `commit()` 명령어를 이용하면 된다. 참고로 select처럼 데이터를 가져오기만 하는 질의문의 경우에는 `commit()`가 필요없다. 데이터에 아무런 변경사항이 없었기 때문이다.

In [5]:
conn.commit()   # commit()은 cursor의 메소드가 아니라 connection의 메소드입니다. 

이제 DB Browser에서 위에 인서트한 데이터가 정상적으로 조회되는지 아래 그림처럼 다시 확인해보자.

<img src="./image/sqlite3.png" alt="sqlite" />

이렇든 `commit()`을 통해 데이터베이스에 데이터 변경이 실제적으로 반영되었다. 이런 것을 데이터베이스에서는 `트랜잭션(transaction)` 관리라고 한다.

트랜잭션의 개념에 대해 아래 글을 읽자.

[트랜잭션이란](https://sjh836.tistory.com/11)

`commit()`을 완료했다면 DB와 대화하는 것을 마무리 지어야한다. 대화의 마무리는 DB와의 연결을 끊는 것으로 이루어진다. 바로 `close()`를 이용하면 된다.

In [6]:
c.close()      # 먼저 커서를 닫은 후
conn.close()    # DB 연결을 닫아 줍니다.

### DDL문으로 테이블 생성하기
---
다양한 테이블을 조회해 보기 앞서, 실제 예제 테이블들을 한번 생성해 보도록 하겠다. 테이블명과 컬럼명을 한글로도 지정할 수 있다.

In [7]:
import sqlite3
import os
db_path = os.getenv('HOME')+'/aiffel/sql_to_db/sqlite/mydb.db'

conn = sqlite3.connect(db_path)
c = conn.cursor()

#- ! 재실행 시 테이블이 존재할 수 있으므로 아래처럼 해당 테이블들을 모두 지워줍니다.
c.execute("DROP TABLE IF EXISTS 도서대출내역")
c.execute("DROP TABLE IF EXISTS 도서대출내역2")
c.execute("DROP TABLE IF EXISTS 대출내역")
c.execute("DROP TABLE IF EXISTS 도서명")


#----- 1st table : 도서대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역 (ID varchar, 이름 varchar, 도서ID varchar, 대출일 varchar, 반납일 varchar)")
#- 생성(create)문 : 테이블명, 변수명, 변수타입을 지정

data = [('101','문강태','aaa','2020-06-01','2020-06-05'),
             ('101','문강태','ccc','2020-06-20','2020-06-25'),
             ('102','고문영','bbb','2020-06-01',None),
             ('102','고문영','ddd','2020-06-08',None),
             ('103','문상태','ccc','2020-06-01','2020-06-05'),
             ('104','강기둥',None,None,None)]
#- 입력할 데이터를 그대로 입력 (변수명 순서 기준대로)

c.executemany('INSERT INTO 도서대출내역 VALUES (?,?,?,?,?)', data)
#- 입력할 데이터를 실제 테이블에 insert하기
#-----------------------------------------------#


#----- 2nd table : 도서대출내역2 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역2 (ID varchar, 이름 varchar, 대출년월 varchar, 대출일수 varchar)")

data = [('101','문강태','2020-06','20일'),
             ('102','고문영','2020-06','10일'),
             ('103','문상태','2020-06','8일'),
             ('104','강기둥','2020-06','3일')]
c.executemany('INSERT INTO 도서대출내역2 VALUES (?,?,?,?)', data)
#--------------------------------------------------#


#----- 3rd table : 대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 대출내역 (ID varchar, 이름 varchar, 도서ID varchar)")

data = [('101','문강태','aaa'),
             ('102','고문영','bbb'),
             ('102','고문영','fff'),
             ('103','문상태','ccc'),
             ('104','강기둥',None)]
c.executemany('INSERT INTO 대출내역 VALUES (?,?,?)', data)
#-----------------------------------------#


#----- 4th table : 도서명 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#

conn.commit()
conn.close()

데이터의 생성/삭제/갱신 등을 처리할 때 필요한 __DML문__의 호출 방법에 대해 바로 위에서 다루었던 코드를 중심으로 가볍게 이해해 보도록 하자.

```python
#----- 4th table : 도서명 -----#

# [1]
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

# [2]
data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

# [3]
c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#
```

__[1]번__의 `CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)` 코드를 주목하자. 이 부분은 "만약 [테이블]이 존재하지 않으면 [테이블]을 생성달라. 그리고 변수명과 각 변수의 타입은 다음과 같이 설정해줘라"라는 의미이다. 예제에서 [테이블]은 [도서명] 테이블이다.

__[2]번__의 코드는 각 변수명에 맞게 데이터를 실제로 생성하는 것이다.

마지막 __[3]번__에서 `INSERT INTO 도서명 VALES (?, ?)` 부분은 각 테이블의 변수(`도서ID`, `도서명`)에 [2]번에서 입력한 데이터(`data`)를 넣겠다는 뜻이다. 이렇게 `?`를 사용해서 데이터를 쿼리에 바인딩하는 기법을 매우 흔히 사용하니 잘 알아두자.<br>
또 하나 눈에 띄는 부분은, `c.executemany()` 메소드이다. 이것은 한꺼번에 여러 개의 데이터 처리를 가능하게 해준다. 입력해야 할 데이터를 위와 같이 `list`로 관리하고 있는 경우 매우 유용하다.

제대로 만들어졌는지 조회해보자.

In [8]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서명'):
  print(row)

('aaa', '악몽을 먹고 자란 소년')
('bbb', '좀비아이')
('ccc', '공룡백과사전')
('ddd', '빨간구두')
('eee', '잠자는 숲속의 미녀')


이제 직접 SQL문을 조작해서 만든 테이블을 조회해보자.

## 3. SQL (1) SQL의 기본

### SQL의 기본
__SQL__이란 무엇을 의미하는 것일까?

SQL의 약자는 Structured Query Language이다. 구조화된… 쿼리… 언어?

조금 더 풀어서 써보자면 데이터베이스(DB)에서 데이터를 조회하고자 할 때 필요한 컴퓨터 언어라고 할 수 있다.

DB라는 공간에 ‘정형화된’(일정한 형식으로 수집되는) 데이터가 차곡차곡 저장되어 있다. 이러한 DB를 특정 언어로 조회해서 가져오는데, 그때 사용하는 언어가 바로 SQL이다.

테이블을 삽입하거나 삭제, 갱신, 조회하는 등 본인이 원하는 형태로 데이터를 만들어 가져올 수 있다. 이번에는 주로 데이터를 조회하고 조작하는 부분에 대해 다뤄보도록 하겠다.

<img src="./image/sql.png" alt="SQL" />

위와 같이 DB에서 특정 테이블(`도서대출내역`)을 조회하고 그것을 가지고 올 때 사용한 `SELECT * FROM 도서대출내역` 이 바로 SQL아다.

자, 이제 쿼리의 기본 구조를 살펴보자.

#### 쿼리의 기본 구조
* `SELECT ~` : 조회할 컬럼명을 선택
* `FROM ~`: 조회할 테이블명을 지정 (위치와 테이블명을 입력)
* `WHERE ~` : 질의할 때 필요한 조건을 설정
* `GROUP BY ~` : 특정 컬럼을 기준으로 그룹을 지어 출력
* `ORDER BY ~` : SELECT 다음에 오는 컬럼 중 정렬이 필요한 부분을 정렬 (기본 설정 : 오름차순)
* `LIMIT 숫자` : Display하고자 하는 행의 수를 설정

위의 기본 구조를 좀 더 풀어서 설명해보겠다.

<img src="./image/sql2.png" alt="SQL" />

<img src="./image/sql3.png" alt="SQL" />

아래 테이블 예시를 가지고 직접 살펴보도록 하자.

#### < 테이블 명 : 도서대출내역 >

<img src="./image/table.png" alt="table" />

테이블을 보시면 도서관에서 관리하고 있는 DB라는 점을 알 수 있다. 그리고 사람들이 각자 어떤 도서를 대출했는지, 언제 대출/반납했는지를 파악할 수가 있다.

* `ID` : 대출을 한 사람의 ID
* `이름` : 대출자의 이름
* `도서ID` : 대출한 도서의 ID
* `대출일` : 도서를 대출한 날짜
* `반납일` : 도서를 반납한 날짜

자세히 보시면, 이렇게 5개의 __열;컬럼__(column)이 있다. 반대로 가로 줄은 __행;로우__(row)라고 부르죠.

<img src="./image/column.png" alt="column" />
<center> [(예시) ID 컬럼(column)] </center>
<img src="./image/row.png" alt="row" />
<center> [(예시) 첫 번째 행(row)] </center>

여기까지는 어렵지 않게 이해가 되었다. 다음으로 넘어가 보자.

지금까지 설명한 기본 구조를 위의 테이블을 활용해 검증하자.

In [10]:
#- 참고 : 실제 조회를 해보시려면 아래처럼 쓰시면 됩니다.
#- c.execute() 괄호 안에 SQL문을 넣으시면 됩니다.

import os
db_path = os.getenv('HOME')+'/aiffel/sql_to_db/sqlite/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서대출내역'):
    print(row)

#- ! 출력 시 'NULL' 대신 'None'으로 출력될 수 있으나 동일하게 이해하시면 됩니다.

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


그럼 이번에는 sqlite DB Browser를 이용해 쿼리를 수행해보자.

#### /* 전체 조회 */

```sql
SELECT * FROM 도서대출내역;
```

<img src="./image/table.png" alt="table" />

`SELECT`와 `FROM` 사이에는 특정 컬럼을 넣어 출력하곤 하는데, 위의 쿼리처럼 별(`*`)을 입력하게 되면 '테이블 전체를 다 가져와라'라는 명령어가 된다.

그럼 특정 컬럼을 지정해보자

#### /* 특정 컬럼을 지정 */

```sql
SELECT ID FROM 도서대출내역;
```

<img src="./image/column.png" alt="column" />

`SELECT`와 `FROM`사이에 ‘`ID`’를 넣으면 '전체 테이블 중에 `ID` 컬럼만 가져와라'라는 명령어가 된다.